In [ ]:
!pip install -qU datasets transformers evaluate optimum[onnxruntime]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install -q optimum[exporters,onnxruntime]==1.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.7/417.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
  Attempting uninstall: optimum
    Found existing installation: optimum 1.23.3
    Uninstalling optimum-1.23.3:
      Successfully uninstalled optimum-1.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.

In [ ]:
# getting the dataset
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_csv('df_cleaned.csv', index_col=0)
df.head()

,text,label
0,"i have a 3g iphone. after 3 hrs tweeting at #riseaustin, it was dead! i need to upgrade. plugin stations at #sxsw.",Negative emotion
1,"know about ? awesome ipadiphone app that you'll likely appreciate for its design. also, they're giving free ts at #sxsw",Positive emotion
2,can not wait for #ipad 2 also. they should sale them down at #sxsw.,Positive emotion
3,i hope this year's festival isn't as crashy as this year's iphone app. #sxsw,Negative emotion
4,"great stuff on fri #sxsw: marissa mayer (google), tim o'reilly (tech booksconferences) & matt mullenweg (wordpress)",Positive emotion


In [ ]:
label2id = {
    "Negative emotion": 0,
    "No emotion toward brand or product": 1,
    "Positive emotion": 2
  }

In [ ]:
# converting it to huggingface dataset
from datasets import DatasetDict, Dataset

hf_dataset = Dataset.from_pandas(df, preserve_index=False)
hf_dataset = hf_dataset.class_encode_column("label")
hf_dataset

Casting to class labels:   0%|          | 0/5821 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 5821
})

In [ ]:
df_mini = df.iloc[:1000,:].copy()
df_mini

,text,label
0,"i have a 3g iphone. after 3 hrs tweeting at #riseaustin, it was dead! i need to upgrade. plugin stations at #sxsw.",Negative emotion
1,"know about ? awesome ipadiphone app that you'll likely appreciate for its design. also, they're giving free ts at #sxsw",Positive emotion
2,can not wait for #ipad 2 also. they should sale them down at #sxsw.,Positive emotion
3,i hope this year's festival isn't as crashy as this year's iphone app. #sxsw,Negative emotion
4,"great stuff on fri #sxsw: marissa mayer (google), tim o'reilly (tech booksconferences) & matt mullenweg (wordpress)",Positive emotion
...,...,...
995,apple set to open popup shop in core of #sxsw action aapl,No emotion toward brand or product
996,apple set to open popup shop in core of sxsw action - they're going to sell ipad2 at #sxsw!,Positive emotion
997,apple set to open popup shop in core of sxsw action #sxsw #apple: apple set to open popup sh...,No emotion toward brand or product
998,apple set to open popup shop in core of sxsw action. leases downtown space during festival #sxsw #apple #austin,No emotion toward brand or product


In [ ]:
hfmini_dataset = Dataset.from_pandas(df_mini, preserve_index=False)
hfmini_dataset = hfmini_dataset.class_encode_column("label")
hfmini_dataset

Casting to class labels:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})

# Vanilla Model

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline
import transformers

model_id = "Akirami/twitter-roberta-sentiment-analysiss-lr-1e-5"

tokenizer = transformers.RobertaTokenizer.from_pretrained(model_id)
classifier = pipeline("text-classification", model="Akirami/twitter-roberta-sentiment-analysiss-lr-1e-5", tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
import evaluate

task_type = "text-classification"

metric = evaluate.load("accuracy")
evaluator = evaluate.evaluator(task_type)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def evaluate_pipeline(pipeline):
    results = evaluator.compute(
        model_or_pipeline=pipeline,
        data=hfmini_dataset,
        metric=metric,
        label_column="label",
        label_mapping=label2id,
    )
    return results



In [ ]:
print("*** Original model")
results = evaluate_pipeline(classifier)
print(results)

*** Original model
{'accuracy': 0.599, 'total_time_in_seconds': 147.81540211999993, 'samples_per_second': 6.765194869125865, 'latency_in_seconds': 0.14781540211999994}


# ONNX Model

In [ ]:
# ONNX
import transformers
from optimum.onnxruntime import ORTModelForSequenceClassification
from optimum.pipelines import pipeline

# importing model and tokenizer
model = ORTModelForSequenceClassification.from_pretrained(
    model_id, export=True # from_transformers=True
)
tokenizer = transformers.RobertaTokenizer.from_pretrained(model_id)

# this also works
# classifier_onnx = pipeline(task="text-classification", model=model_id, tokenizer=tokenizer,  accelerator="ort")

# saving the model and tokenizer
model.save_pretrained("./model_onnx")
tokenizer.save_pretrained("./model_onnx")

# creating a pipeline
classifier_onnx = pipeline(task_type, model=model, tokenizer=tokenizer) # we do not need to provide accelerator="ort", it will be by default

The argument `from_transformers` is deprecated, and will be removed in optimum 2.0.  Use `export` instead
Framework not specified. Using pt to export the model.
Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: RobertaForSequenceClassification *****
Using framework PyTorch: 2.5.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False


In [ ]:
results = evaluate_pipeline(classifier_onnx)
print("*** ONNX")
print(results)

*** ONNX
{'accuracy': 0.599, 'total_time_in_seconds': 118.6795074470001, 'samples_per_second': 8.426054518692538, 'latency_in_seconds': 0.11867950744700011}


# ONNX Optimized Model

**Optimizing and quantizing in pipelines**

The pipeline() function can not only run inference on vanilla ONNX Runtime checkpoints - you can also use checkpoints optimized with the `ORTQuantizer` and the `ORTOptimizer`.

In [ ]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# loading the onnx model into the optimizer
optimizer = ORTOptimizer.from_pretrained(model)

# applying optimization and savin the model
optimizer.optimize(
    OptimizationConfig(
        optimization_level=99, # 1, 2 or 99
    ),
    save_dir="./model_onnx",
)

# loading the optimized onnx model
model_optimized = ORTModelForSequenceClassification.from_pretrained(
    "./model_onnx", file_name="model_optimized.onnx"
)

# calling the pipeline object on it
classifier_optimized = pipeline(task_type, model=model_optimized, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/optimum/onnxruntime/configuration.py:770: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in model_onnx/ort_config.json
Optimized model saved at: model_onnx (external data format: False; saved all tensor to one file: True)


In [ ]:
results = evaluate_pipeline(classifier_optimized)
print("*** ONNX optimizer")
print(results)

*** ONNX optimizer
{'accuracy': 0.599, 'total_time_in_seconds': 103.057864376, 'samples_per_second': 9.703286654103021, 'latency_in_seconds': 0.10305786437600001}


# ONNX Quantized Model

Checking File Sizes Before Quantization

In [ ]:
!ls -lh *

-rw-r--r-- 1 root root 725K Nov 11 04:03 df_cleaned.csv

model_onnx:
total 953M
-rw-r--r-- 1 root root 1004 Nov 11 04:56 config.json
-rw-r--r-- 1 root root 446K Nov 11 04:51 merges.txt
-rw-r--r-- 1 root root 476M Nov 11 04:51 model.onnx
-rw-r--r-- 1 root root 476M Nov 11 04:56 model_optimized.onnx
-rw-r--r-- 1 root root 1.3K Nov 11 04:56 ort_config.json
-rw-r--r-- 1 root root  958 Nov 11 04:51 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Nov 11 04:51 tokenizer_config.json
-rw-r--r-- 1 root root 976K Nov 11 04:51 vocab.json

sample_data:
total 55M
-rwxr-xr-x 1 root root 1.7K Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root 295K Nov  7 20:56 california_housing_test.csv
-rw-r--r-- 1 root root 1.7M Nov  7 20:56 california_housing_train.csv
-rw-r--r-- 1 root root  18M Nov  7 20:56 mnist_test.csv
-rw-r--r-- 1 root root  35M Nov  7 20:56 mnist_train_small.csv
-rwxr-xr-x 1 root root  962 Jan  1  2000 README.md


We see that the `model.onnx` and `model_optimized.onnx` are of same size, i.e. 476MB

In [ ]:
!du -sh *

728K	df_cleaned.csv
953M	model_onnx
55M	sample_data


In [ ]:
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# loading the onnx model into ORT Quantizer
quantizer = ORTQuantizer.from_pretrained(model)

# creating a quantization config - int8 quantization
qconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=True)

# quantizing and saving the onnx model - applies dynamiz quantization
quantizer.quantize(save_dir="./model_onnx", quantization_config=qconfig)

# loading the quantized model into ORT
model_quantized = ORTModelForSequenceClassification.from_pretrained(
    "./model_onnx", file_name="model_quantized.onnx"
)

# creating a pipeline from the quantized onnx model
classifier_quantized = pipeline(task_type, model=model_quantized, tokenizer=tokenizer)

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: True)
Quantizing model...
Saving quantized model at: model_onnx (external data format: False)
Configuration saved in model_onnx/ort_config.json


In [ ]:
!ls -lh *

-rw-r--r-- 1 root root 725K Nov 11 04:03 df_cleaned.csv

model_onnx:
total 1.1G
-rw-r--r-- 1 root root  969 Nov 11 05:04 config.json
-rw-r--r-- 1 root root 446K Nov 11 04:51 merges.txt
-rw-r--r-- 1 root root 476M Nov 11 04:51 model.onnx
-rw-r--r-- 1 root root 476M Nov 11 04:56 model_optimized.onnx
-rw-r--r-- 1 root root 121M Nov 11 05:04 model_quantized.onnx
-rw-r--r-- 1 root root  829 Nov 11 05:04 ort_config.json
-rw-r--r-- 1 root root  958 Nov 11 04:51 special_tokens_map.json
-rw-r--r-- 1 root root 1.3K Nov 11 04:51 tokenizer_config.json
-rw-r--r-- 1 root root 976K Nov 11 04:51 vocab.json

sample_data:
total 55M
-rwxr-xr-x 1 root root 1.7K Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root 295K Nov  7 20:56 california_housing_test.csv
-rw-r--r-- 1 root root 1.7M Nov  7 20:56 california_housing_train.csv
-rw-r--r-- 1 root root  18M Nov  7 20:56 mnist_test.csv
-rw-r--r-- 1 root root  35M Nov  7 20:56 mnist_train_small.csv
-rwxr-xr-x 1 root root  962 Jan  1  2000 README.md


We see that the `model_quantized.onnx` got shrunk to 121MB, which approximately 4 times smaller

In [ ]:
results = evaluate_pipeline(classifier_quantized)
print("*** ONNX quantizer")
print(results)

*** ONNX quantizer
{'accuracy': 0.614, 'total_time_in_seconds': 77.17517084600013, 'samples_per_second': 12.957535293254598, 'latency_in_seconds': 0.07717517084600013}


# Saving the quantized ONNX model

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model_quantized.save_pretrained("twitter-roberta-sentiment-analysiss-onnx-quantized")
model_quantized.push_to_hub("twitter-roberta-sentiment-analysiss-onnx-quantized", repository_id="Akirami/twitter-roberta-sentiment-analysiss-onnx-quantized", use_auth_token=True)

model_quantized.onnx:   0%|          | 0.00/126M [00:00<?, ?B/s]

Missealneous

In [ ]:
from optimum.onnxruntime import AutoOptimizationConfig
from optimum.onnxruntime.configuration import OptimizationConfig

optimization_config = AutoOptimizationConfig.O2()
optimization_config

/usr/local/lib/python3.10/dist-packages/optimum/onnxruntime/configuration.py:770: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(


OptimizationConfig(optimization_level=2, optimize_for_gpu=False, fp16=False, optimize_with_onnxruntime_only=None, enable_transformers_specific_optimizations=True, disable_gelu=None, disable_gelu_fusion=False, disable_layer_norm=None, disable_layer_norm_fusion=False, disable_attention=None, disable_attention_fusion=False, disable_skip_layer_norm=None, disable_skip_layer_norm_fusion=False, disable_bias_skip_layer_norm=None, disable_bias_skip_layer_norm_fusion=False, disable_bias_gelu=None, disable_bias_gelu_fusion=False, disable_embed_layer_norm=True, disable_embed_layer_norm_fusion=True, enable_gelu_approximation=False, use_mask_index=False, no_attention_mask=False, disable_shape_inference=False, use_multi_head_attention=False, enable_gemm_fast_gelu_fusion=False, use_raw_attention_mask=False, disable_group_norm_fusion=True, disable_packed_kv=True, disable_rotary_embeddings=False)

In [ ]:
from optimum.pipelines import pipeline

onx_cls = pipeline(task_type, model="Akirami/twitter-roberta-sentiment-analysiss-onnx-quantized", tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/977 [00:00<?, ?B/s]

model_quantized.onnx:   0%|          | 0.00/126M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [ ]:
results = evaluate_pipeline(onx_cls)
print("*** ONNX quantizer")
print(results)

*** ONNX quantizer
{'accuracy': 0.614, 'total_time_in_seconds': 80.27215218699985, 'samples_per_second': 12.457620392068558, 'latency_in_seconds': 0.08027215218699985}


In [ ]:
onx_cls2 = pipeline(task_type, model="Akirami/twitter-roberta-sentiment-analysiss-onnx-quantized")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
results = evaluate_pipeline(onx_cls2)
print("*** ONNX quantizer")
print(results)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


*** ONNX quantizer
{'accuracy': 0.614, 'total_time_in_seconds': 69.85501699399993, 'samples_per_second': 14.315364064486495, 'latency_in_seconds': 0.06985501699399993}
